In [7]:
import cv2
import numpy as np
from numpy import array
import pickle
import os
import pandas as pd
mapped_values = {0:0, 25:1, 75:2}

# LOADS AUGMENTED IMAGES 

-- may need to change the path if running locally

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
root_directory = 'gdrive/My Drive/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [8]:
root_directory = "/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/CompiledCode/TSD/"

# Annotations Classes Creates P Files for the data

In [9]:
class Annotations:
    def __init__(self, path, et, data_tp):
        self.pathInput = path
        self.effect_type = et
        self.effects = self.effect_type.split("_")
        self.data_type = data_tp
        self.compile_csv()
        self.data_frame = self.process_csv()
        self.create_annotations(mapped_values)

    def compile_csv(self):
        all_csv = list()
        extension = 'csv'
        print(self.pathInput)
        for subdir, dirs, files in os.walk(self.pathInput):
            print(subdir)
            for file in files:
                if file.endswith(extension):
                    all_csv.append(subdir + "/" + file)
        outname = self.effect_type + "combined.csv"
        combined_csv = pd.concat([pd.read_csv(f) for f in all_csv])
        fullname = os.path.join(self.pathInput, outname)
        if os.path.exists(fullname):
            os.remove(fullname)
        combined_csv.to_csv(fullname, index=False, encoding='utf-8-sig')
        return fullname

    def process_csv(self):
        print(original_path + effect_type)
        df = pd.read_csv(original_path + effect_type + "combined.csv")
        df = df.sort_values(by=['Unnamed: 0'])
        df = df.drop_duplicates()
        print(len(df))
        labelled_class = []
        labelled_class_details = []
        all_files = set()

        for file in os.listdir(original_path):
            if file.endswith(".png"):
                all_files.add(file)
                labelled_class.append(self.effect_type)
                if '-0' in file:
                    labelled_class_details.append('NONE')
                elif '-25' in file:
                    labelled_class_details.append(self.effect_type + "_LOW")
                elif '-75' in file:
                    labelled_class_details.append(self.effect_type + "_HIGH")
        all_files = sorted(all_files)
        print(len(all_files))
        print(len(labelled_class))
        
        
        df['class'] = labelled_class
        for i, column in df.iterrows():
            img_id = column['Unnamed: 0']
            for x in all_files:
                if x.startswith(img_id):
                    df.at[i,'full_img_pth'] = x
        df_select = df
        #df_select = df_select.fillna(0)
        df_select.to_csv(original_path + effect_type + "ALLcombined.csv", index=False)
        return df

    def create_annotations(self, mapped_values):
        X = []
        y = []
        y_q_f = []
        full_paths = []
        classes_compiled = []

        for row, c in self.data_frame.iterrows():
            current_class = c[self.effects[0]]
            img_pth = self.pathInput + c['full_img_pth']

            if os.path.exists(img_pth):
                x_image = cv2.imread(img_pth)
                x_image = cv2.resize(x_image, dsize=(32, 32))
                sign_label = c['SignType']
                X.append(x_image)
                y.append(sign_label)
                actual_label = mapped_values.get(current_class)
                y_q_f.append(actual_label)
                full_paths.append(img_pth)
                classes_compiled.append(c['class'])
        print(len(classes_compiled))
        X = array (X)
        y = array (y)
        y_q_f = array(y_q_f)
        data = {"data":X,
                "labels":y,
                "quality_data_label": y_q_f,
                "full_paths": full_paths,
                "classes": classes_compiled}
        pickle_out = open(self.pathInput + "{}_{}.p".format(self.effect_type, self.data_type),"wb")
        pickle.dump(data, pickle_out)
        pickle_out.close()
        return X, y, y_q_f


In [10]:
if __name__ == '__main__':
    base_path = root_directory + "/IMAGES/"
    desired_effects = []
    for directory in os.listdir(base_path):
        if os.path.isdir(base_path + directory):
            desired_effects.append(directory)
    print(len(desired_effects))
    desired_effects = sorted(desired_effects)
    for element in desired_effects:
        print("PROCESSING {}".format(element))
        effect_type = element
        dt = "RANDOM"
        original_path = root_directory +  "IMAGES/{}/" \
            .format(effect_type)
        first_file = original_path +  effect_type + "combined.csv"
        second_file = original_path +  effect_type + "ALLcombined.csv"
        if os.path.exists(first_file):
            os.remove(first_file)
        if os.path.exists(second_file):
            os.remove(second_file)
        a = Annotations(original_path, effect_type, dt)



19
PROCESSING BACKLIGHTSUN
/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/CompiledCode/TSD/IMAGES/BACKLIGHTSUN/
/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/CompiledCode/TSD/IMAGES/BACKLIGHTSUN/
/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/CompiledCode/TSD/IMAGES/BACKLIGHTSUN/BACKLIGHTSUN
6000
6000
6000
6000
PROCESSING BACKLIGHTSUN_DIRTLENS
/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/CompiledCode/TSD/IMAGES/BACKLIGHTSUN_DIRTLENS/
/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/CompiledCode/TSD/IMAGES/BACKLIGHTSUN_DIRTLENS/
/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/CompiledCode/TSD/IMAGES/BACKLIGHTSUN_DIRTLENS/BACKLIGHTSUN_DIRTLENS
1000
1000
1000
1000
PROCESSING BACKLIGHTSUN_DIRTLENS_MOTIONBLUR
/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/CompiledCode/TSD/IMAGES/BACKLIGHTSUN_DIRTLENS_MOTIONBLUR/
/Users/luciaeve/Documents/EMSE/KAISERSLAUTERN/THESIS/code/CompiledCode/TSD/IMAGES/BACKLIGHTSUN_DIRTLENS_

In [11]:
import os
import pandas as pd
import pickle
from numpy import array

def compile_csv(path_input):
    all_csv = list()
    extension = 'csv'
    for subdir, dirs, files in os.walk(path_input):
        for file in files:
            if file.endswith(extension) and "ALLcombined" in file:
                all_csv.append(subdir + "/" + file)
    outname = "ALL_combined.csv"
    combined_csv = pd.concat([pd.read_csv(f) for f in all_csv])
    fullname = os.path.join(path_input, outname)
    if os.path.exists(fullname):
        os.remove(fullname)
    combined_csv.to_csv(fullname, index=False, encoding='utf-8-sig')
    return fullname

def combine_p_values(pathInput):
    final_data = {"data":[],
                  "labels":[],
                  "quality_data_label": [],
                  "full_paths":[],
                  "classes":[]}
    for subdir, dirs, files in os.walk(pathInput):
        #print(subdir)
        for file in files:
            #print(file)
            if file.endswith('.p') and 'VALIDATION' not in file and 'RANDOM' in file:
                print(file)
                with open (subdir + "/" + file, mode='rb') as f:
                    test_data = pickle.load(f)

                for key, value in test_data.items() :
                    print (key)
                final_data['data'].extend(test_data['data'])
                final_data['labels'].extend(test_data['labels'])
                final_data['quality_data_label'].extend(test_data['quality_data_label'])
                final_data['full_paths'].extend(test_data['full_paths'])
                final_data['classes'].extend( test_data['classes'])
    outname = "VALIDATION.p"
    fullname = os.path.join(fp, outname)
    pickle_out = open(fullname,"wb")

    ## changes these to arrays
    final_data['data'] = array(final_data['data'])
    final_data['labels'] = array(final_data['labels'])
    final_data['quality_data_label'] = array(final_data['quality_data_label'])
    print((final_data['data'].shape))
    pickle.dump(final_data, pickle_out)
    pickle_out.close()


fp = root_directory + "IMAGES/"
compile_csv(fp)
df = pd.read_csv(fp + "ALL_combined.csv")
df_select = df.fillna(0)
df_select.to_csv(fp + "ALL_combined.csv", index=False)
combine_p_values(fp)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


BACKLIGHTSUN_DIRTLENS_RANDOM.p
data
labels
quality_data_label
full_paths
classes
BACKLIGHTSUN_MOTIONBLUR_RANDOM.p
data
labels
quality_data_label
full_paths
classes
DARKNESS_RAIN_RANDOM.p
data
labels
quality_data_label
full_paths
classes
RAIN_RANDOM.p
data
labels
quality_data_label
full_paths
classes
DARKNESS_DIRTLENS_MOTIONBLUR_RANDOM.p
data
labels
quality_data_label
full_paths
classes
MOTIONBLUR_RANDOM.p
data
labels
quality_data_label
full_paths
classes
DARKNESS_MOTIONBLUR_RAIN_RANDOM.p
data
labels
quality_data_label
full_paths
classes
DIRTLENS_RAIN_RANDOM.p
data
labels
quality_data_label
full_paths
classes
DARKNESS_DIRTLENS_RAIN_RANDOM.p
data
labels
quality_data_label
full_paths
classes
BACKLIGHTSUN_DIRTLENS_MOTIONBLUR_RANDOM.p
data
labels
quality_data_label
full_paths
classes
DARKNESS_DIRTLENS_RANDOM.p
data
labels
quality_data_label
full_paths
classes
DARKNESS_MOTIONBLUR_RANDOM.p
data
labels
quality_data_label
full_paths
classes
BACKLIGHTSUN_RANDOM.p
data
labels
quality_data_label
f